In [1]:
import requests
#from main import *


class FlightAndPassengerInformation:
    '''Get The Passenger Information via User Input'''

    def __init__(self):
        self.flightNumber = input("Enter Flight Number: ")
        self.customerFirstName = input("Enter Passenger First Name: ")
        self.customerLastName = input("Enter Passenger Last Name: ")
        self.flightDate = input("Enter flight date YYYY-MM-DD: ")

        self.delayURL = "https://aerodatabox.p.rapidapi.com/flights/number" + \
                        "/" + self.flightNumber + "/" + self.flightDate        
        print(self.delayURL)
        self.headers = {
            'x-rapidapi-key': "",
            'x-rapidapi-host': "aerodatabox.p.rapidapi.com"
        }
        self.request = requests.request("GET", self.delayURL, headers=self.headers)
        self.response = self.request.text
       



In [8]:
import sqlite3
import random


class Database(object):
    """sqlite3 database class that holds testers jobs"""
    DB_LOCATION = "wallet.sqlite"

    def __init__(self, db_location=None):
        """Initialize db class variables"""
        if db_location is not None:
            self.connection = sqlite3.connect(db_location)
        else:
            self.connection = sqlite3.connect(self.DB_LOCATION)
        self.cur = self.connection.cursor()


def generate_transaction_number():
    db = Database()
    query = '''SELECT transaction_ID from transactions;'''
    query_result = db.cur.execute(query)
    transaction_ids = []
    for row in query_result:
        transaction_ids.append(row[0])
    m = 100000000
    transaction_num = random.choice([x for x in range(m) if x not in transaction_ids])
    return transaction_num


def generate_customer_ID():
    db = Database()
    query = '''SELECT customer_id from transactions;'''
    query_result = db.cur.execute(query)
    # db.connection.commit()
    # db.connection.row_factory = sqlite3.Row
    customer_id = []
    for row in query_result:
        customer_id.append(row[0])
    m = 100000000
    customer_num = random.choice([x for x in range(m) if x not in customer_id])
    return customer_num


def write_customer(transaction_id, first, last, flight_number, flight_delay,
                   calculated_payout, customer_id, flight_date):
    db = Database()

    db.cur.execute('''INSERT INTO transactions (transaction_ID, customer_first_name, customer_last_name,
                         flight_number, flight_delay_amt,
                           calculated_payout, customer_id, flight_date)
                          VALUES (?,?,?,?,?,?,?,?);''', (
        transaction_id, first, last, flight_number, flight_delay, calculated_payout,
        customer_id, flight_date))
    db.connection.commit()
    # # db.connection.row_factory = sqlite3.Row
    # customer_id = []
    # for row in query_result:
    #     customer_id.append(row[0])
    # m = max(customer_id)
    # customer_num = random.choice([x for x in range(m) if x not in customer_id])
    # return customer_num


def get_wallet_amount():
    db = Database()
    query = '''SELECT dollar_amount from wallet where customer_id = (SELECT MAX(customer_id) FROM wallet);'''
    query_result = db.cur.execute(query)
    transaction_ids = []
    for row in query_result:
        transaction_ids.append(row[0])
    wallet_amount = transaction_ids[0]
    return wallet_amount


def calculate_payout(delay_time):
    delay_rate = 25  # perhour
    if delay_time <= 0:
        return 0
    else:
        payout = delay_rate * delay_time/60
        return payout


def get_wallet_change(payout):
    new_value = get_wallet_amount() - payout
    return new_value


def update_insurance_wallet(dollar_amount, transactionID):
    db = Database()
    db.cur.execute('''insert into wallet ( dollar_amount, transactionID)
                    VALUES ( ?, ?);''', (dollar_amount, transactionID))
    db.connection.commit()




In [9]:
import json
import ast
import datetime

if __name__ == '__main__':

    information = FlightAndPassengerInformation()

    transaction_num = generate_transaction_number()
    customer_id = generate_customer_ID()
    

    info = json.loads(information.response)
    info = info[0]
    for k,v in info['arrival'].items():
        if k == "scheduledTimeLocal":
            s = v
        if k == "actualTimeLocal":
            a = v
        else:
            pass

    date_time_obj = datetime.datetime.strptime(a, '%Y-%m-%d %H:%M-%S:%f')
    date_time_obj2 = datetime.datetime.strptime(s, '%Y-%m-%d %H:%M-%S:%f')

    timedelta = date_time_obj-date_time_obj2

    td = (timedelta.total_seconds())
    calcDelay = td/60
    if calcDelay <= 0:
        calcDelay = 0

    write_customer(transaction_num, information.customerFirstName, information.customerLastName,
                    information.flightNumber, calcDelay, calculate_payout(calcDelay), customer_id, information.flightDate)

    print("passenger delayed (min): ", calcDelay, " passenger_owed: $", calculate_payout(calcDelay))

    update_insurance_wallet(get_wallet_change(calcDelay), transaction_num)


Enter Flight Number:  \\
Enter Passenger First Name:  
Enter Passenger Last Name:  
Enter flight date YYYY-MM-DD:  


https://aerodatabox.p.rapidapi.com/flights/number/\\/


KeyError: 0

In [ ]:
def show_wallet():
    db = Database()
    query = '''SELECT * FROM wallet;'''
    query_result = db.cur.execute(query)
    wallet_rows = []
    for row in query_result:
        wallet_rows.append(row)
    wallet_amount = wallet_rows
    return wallet_amount


def show_transactions():
    db = Database()
    query = '''SELECT * FROM transactions;'''
    query_result = db.cur.execute(query)
    transactions = []
    for row in query_result:
        transactions.append(row)
    transactions = transactions
    return transactions

In [10]:
import pandas as pd
wallet = pd.DataFrame(show_wallet())
wallet.columns = ["id", "wallet_amount", "cust_id"]

In [11]:
wallet

,id,wallet_amount,cust_id
0,0,500,0
1,1,411,55764155
2,2,411,67422836
3,3,322,79970120
4,4,233,56929802


In [12]:
transactions = pd.DataFrame(show_transactions())
transactions.columns = ["transaction_num","FirstName", "LastName",
                    "flightNumber", "calculated_delay", "calculate_payout", "customer_id", "flight_delay"]
transactions

,transaction_num,FirstName,LastName,flightNumber,calculated_delay,calculate_payout,customer_id,flight_delay
0,55764155,Igor,S,B6402,89,37.083333,50379814,None
1,56929802,Igor,S,B6402,89,37.083333,21139743,2021-05-28
2,67422836,Igor,S,B6402,0,0.000000,60217134,None
3,79970120,Igors,Dog,B6402,89,37.083333,96549666,2021-05-28


In [ ]:
def flightDashboard(depart, arrive):
    st.markdown("## FLIGHT DASHBOARD")
    st.text(f"Current Flights from {departure_iata} to {arrival_iata}")
    api_key = st.secrets["AVIATIONSTACK_API_KEY"]
    aviationstack_api = os.environ["AVIATIONSTACK_API_KEY"]
    aviationstack_url = 'http://api.aviationstack.com/v1/flights'
    params = {
      'access_key': aviationstack_api,
      'dep_iata': departure_iata,
      'arr_iata': arrival_iata,
    }
    api_result = requests.get(aviationstack_url, params)
    api_response = api_result.json()
    for i in range(0, len(api_response['data'])-1):
        airline = api_response['data'][i]['airline']['name']
        est_departure = api_response['data'][i]['departure']['estimated'].split("T")
        departure_delay = api_response['data'][i]['departure']['delay']
        est_arrival = api_response['data'][i]['arrival']['estimated'].split("T")
        arrival_delay = api_response['data'][i]['arrival']['delay']
        flight = api_response['data'][i]['flight']['iata']
        
        st.text(f"Airline Name: {airline}")
        st.text(f"Flight IATA: {flight}")
        st.text(f"Est departure date and time: {est_departure[0]}:{est_departure[1].split('+')[0]}")
        st.text(f"Departure Delay:{departure_delay}")
        st.text(f"Est arrival date and time: {est_arrival[0]}:{est_departure[1].split('+')[0]}")
        st.text(f"Arrival Delay:{arrival_delay}")
        st.text("-----------------------------------------------------")
        
departure_iata = st.text_input("Enter the 3-letter IATA code of the Airport you are departing from.", max_chars=3, type="default")
arrival_iata = st.text_input("Please enter the IATA code of the Destination Airport.", max_chars=3, type="default")

if st.button("Find Flights", key='55'):
    flightDashboard(departure_iata, arrival_iata)
    

In [1]:
import random
from functionsPassenger import *

unique_sequence = uniqueid()
transaction_id = next(unique_sequence)
transaction_id

582564927

In [6]:
import json
from functionsPassenger import *
from eth_account import Account
from web3 import Web3
private_user = '981532905b1c88d2bf703353945496d0fc271ce155d4288d43b4b73903713564'
account_temp = Account.from_key(private_user)

In [7]:
account_temp